<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/perplexityAttempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install torch
!pip install dataclasses
!pip install hydra-core
!pip install sentencepiece

In [ ]:
%%capture
import torch
#xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large')
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large', force_reload=True)
xlmr.eval()

In [ ]:
def force_decode(sentence):
    indices = xlmr.encode(sentence).view(1, -1)    # [B=1 x T]
    feats = xlmr.extract_features(indices)         # [B=1 x T x D]
    #scores = xlmr.model.output_layer(feats)        # [B=1 x T x V]
    scores = xlmr.model.output(feats)
    log_probs = torch.log_softmax(scores, dim=-1)  # [B=1 x T x V]
    lop_probs = log_probs.gather(dim=2, index=indices.unsqueeze(2)) # [B x T x 1]
    log_prob = lop_probs.squeeze(2).sum(dim=1)    # [B=1 x T] -> [B=1]
    return torch.exp(log_prob).item()

sents = ["this is a good sentence",
         "*%&^( some *^(*&)(^&*(^% gibberish &^$^%$^&*&",
         "c'est une phrase française",
         "c'est une phrase (^&*(^% gibberish &^$^% française",         
         "ಇದು ಕನ್ನಡ ವಾಕ್ಯ",
         "ಇದು ಕನ್ನಡ (^&*(^% gibberish &^$^% ವಾಕ್ಯ",
         "यह एक हिंदी वाक्य है",
         "यह एक हिंदी (^&*(^% gibberish &^$^% वाक्य है"]
for sent in sents:
    print('%.6f' % force_decode(sent), sent)

RuntimeError: ignored